In [5]:
require 'daru'
require 'daru_plotting_plotly'
require 'hg'

Daru.plotting_library = :plotly

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

:plotly

In [6]:
class LeastSquareFitting
  def initialize(xs, ys)
    raise ArgumentError "xs and ys should be same size" if xs.size != ys.size
    @xs = xs
    @ys = ys
  end
  
  def fit
    xy = @xs.zip(@ys).map { |x, y| x * y }.sum
    x = @xs.sum
    y = @ys.sum
    x2 = @xs.sum { |x| x ** 2 }
    n = @xs.size
    
    @a = (n*xy - x*y)/(n*x2 - x**2)
    @b = (x2*y - xy*x)/(n*x2 - x**2)
  end
  
  def sigma_y
    @sigma_y ||= Math.sqrt(1.0/(@xs.size - 2)*(@xs.zip(@ys).map {|x, y| (x * @a + @b - y) ** 2}.sum))
  end
  
  def sigma_a
    x = @xs.sum
    x2 = @xs.sum { |x| x ** 2 }
    @sigma_a ||= sigma_y * Math.sqrt(@xs.size.to_f/(@xs.size.to_f*x2 - x**2))
  end
  
  def sigma_b
    x = @xs.sum
    x2 = @xs.sum { |x| x ** 2 }
    @sigma_b ||= sigma_y * Math.sqrt(x2/(@xs.size.to_f*x2 - x**2))
  end
  
  attr_reader :a, :b
end

class ExpFitting < LeastSquareFitting
  def initialize(xs, ys)
    super(xs, ys.map{ |y| Math.log(y) })
  end
  
  def to_s
    "log y = #{'%1.2e' % @a} x #{ @b.positive? ? '+' : '-' } #{'%1.2e' % @b.abs }"
  end
  
  def points(min, max)
    ys = [min, max].map { |x| Math.exp(@b) * Math.exp( @a * x ) }
    { x: [min, max], y: ys, name: to_s }
  end
end

class LinearFitting < LeastSquareFitting
  def points(min, max)
    ys = [min, max].map { |x| x * @a + @b }
    { x: [min, max], y: ys , name: to_s}
  end
  
  def to_s
    "y = #{'%1.2f' % @a} x + #{'%1.2f' % @b}"
  end
end

:to_s

In [17]:
['itano', 'nakagawa', 'sangenya', 'nagura'].each do |member|
  data = ['dark', '5cm', '15cm', '25cm'].map do |name|
    file_name = "data/#{member}/#{name}_solar_battery.csv"
    if File.exists?(file_name)
      Daru::DataFrame.from_csv(file_name).sort(['v']).to_h.map { |k, v| [k == 'v' ? 'x' : 'y', v.to_a] }.to_h.merge(name: "#{member}_#{name}", mode: 'markers+lines')
    end
  end
  Plotly::Plot.new(data: data, layout: { xaxis: { title: '電圧(V)' }, yaxis: { title: '電流(I)' }, font: { size: 18 } }).show
end
nil

#<Plotly::Offline::HTML:0x007fc6d9a05468 @id="8f5d891b-2924-4b10-949d-bd83b56ec44a", @data=[{:y=>[-0.082, -0.0274, -0.0135, -0.011800000000000001, -0.0111, -0.0092, -0.0074, -0.004200000000000001, -0.0024, -0.0012, -0.00096, -0.00067, -0.000407, -0.000321, -0.000261, -0.000127, -7.7e-05, -4.4e-06, 0.0011, 0.0019, 0.0031000000000000003, 0.004, 0.0063, 0.0070999999999999995, 0.008, 0.009800000000000001, 0.010199999999999999, 0.0109], :x=>[-10.2, -3.48, -1.712, -1.506, -1.418, -1.186, -0.95, -0.537, -0.307, -0.1555, -0.1237, -0.0857, -0.0523, -0.0412, -0.0334, -0.0162, -0.0097, -0.0003, 0.142, 0.247, 0.383, 0.466, 0.554, 0.57, 0.588, 0.62, 0.626, 0.636], :name=>"itano_dark", :mode=>"markers+lines"}, {:y=>[-0.0033799999999999998, -0.0033, -0.00317, -0.0029300000000000003, -0.0024100000000000002, -0.0035600000000000002, -0.0025499999999999997, -0.0022, -0.0019, -0.0016, -0.0013000000000000002, -0.001, -0.0008, -0.0006, -0.0004, 0], :x=>[0.34, 0.343, 0.356, 0.357, 0.359, 0.359, 0.362, 0.366, 0.371, 0.375, 0.379, 0.384, 0.387, 0.39, 0.393, 0.422], :name=>"itano_5cm", :mode=>"markers+lines"}, {:y=>[-0.0025, -0.002, -0.0017, -0.0014, -0.0011, -0.0008, -0.0005, -0.00035, 0], :x=>[0.253, 0.302, 0.329, 0.352, 0.372, 0.389, 0.403, 0.409, 0.428], :name=>"itano_15cm", :mode=>"markers+lines"}, {:y=>[-0.0017, -0.0015, -0.0013000000000000002, -0.0011, -0.0009000000000000001, -0.0007, -0.0005, -0.0003, 0], :x=>[0.173, 0.201, 0.222, 0.246, 0.272, 0.293, 0.316, 0.336, 0.37], :name=>"itano_25cm", :mode=>"markers+lines"}], @layout={:xaxis=>{:title=>"電圧(V)"}, :yaxis=>{:title=>"電流(I)"}, :font=>{:size=>18}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fc6da0a9698 @id="4588e63d-df88-4150-92e2-607a4f82f615", @data=[{:y=>[-0.0099, -0.004900000000000001, -0.0016, -0.00147, -0.00127, -0.0011, -0.00092, -0.00082, -0.000618, -0.000472, -0.000226, -0.000126, -3.0e-06, 5.0e-06, 0.000176, 0.0005200000000000001, 0.00062, 0.0007199999999999999, 0.0009699999999999999, 0.0012, 0.0017, 0.0022, 0.0028, 0.0037, 0.0045000000000000005, 0.0066, 0.008199999999999999, 0.0091, 0.01], :x=>[-1.218, -0.617, -0.203, -0.184, -0.159, -0.143, -0.115, -0.103, -0.07790000000000001, -0.0594, -0.028300000000000002, -0.0158, -0.0003, 0.0007, 0.022, 0.06559999999999999, 0.0783, 0.0903, 0.121, 0.146, 0.207, 0.275, 0.348, 0.455, 0.542, 0.704, 0.785, 0.815, 0.844], :name=>"nakagawa_dark", :mode=>"markers+lines"}, {:y=>[-0.0024500000000000004, -0.0021000000000000003, -0.0019, -0.0016, -0.0014, -0.0011, -0.00091, -0.0007, -0.0005, -0.00026, 0], :x=>[0.247, 0.254, 0.258, 0.266, 0.271, 0.277, 0.281, 0.286, 0.292, 0.298, 0.322], :name=>"nakagawa_5cm", :mode=>"markers+lines"}, {:y=>[-0.0024, -0.0021000000000000003, -0.0017, -0.0014, -0.0011, -0.0008, -0.0006, -0.0004, -0.00029, 0], :x=>[0.24, 0.254, 0.272, 0.286, 0.299, 0.312, 0.32, 0.329, 0.334, 0.35], :name=>"nakagawa_15cm", :mode=>"markers+lines"}, {:y=>[-0.0015600000000000002, -0.0014, -0.0012, -0.001, -0.0008, -0.0006, -0.0004, -0.00025, 0], :x=>[0.157, 0.174, 0.195, 0.215, 0.23, 0.256, 0.277, 0.291, 0.314], :name=>"nakagawa_25cm", :mode=>"markers+lines"}], @layout={:xaxis=>{:title=>"電圧(V)"}, :yaxis=>{:title=>"電流(I)"}, :font=>{:size=>18}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fc6da434600 @id="473af6b2-fcca-4f5e-ad0a-0fd1cc6c6eeb", @data=[{:y=>[-0.009000000000000001, -0.0068, -0.005900000000000001, -0.0044, -0.0039, -0.0033, -0.0029, -0.0017, -0.00115, -0.000743, -0.000439, -0.00014, -7.3e-05, -3.3e-05, -8.0e-07, 8.0e-07, 9.6e-05, 0.00023, 0.000226, 0.00041, 0.00059, 0.00077, 0.0013000000000000002, 0.0018000000000000002, 0.0023, 0.0029, 0.0039, 0.0048, 0.0055, 0.006500000000000001, 0.0075, 0.0083, 0.0094], :x=>[-0.991, -0.911, -0.874, -0.802, -0.774, -0.734, -0.709, -0.614, -0.538, -0.458, -0.35, -0.142, -0.07529999999999999, -0.0346, -0.0008, 0.105, 0.123, 0.165, 0.19, 0.267, 0.311, 0.339, 0.39, 0.418, 0.44, 0.459, 0.486, 0.505, 0.519, 0.538, 0.554, 0.565, 0.582], :name=>"sangenya_dark", :mode=>"markers+lines"}, {:y=>[-0.003, -0.0023, -0.0018000000000000002, -0.0014, -0.001, -0.0008, -0.0006, -0.0004, -0.00034, 0], :x=>[0.309, 0.345, 0.36, 0.37, 0.379, 0.384, 0.388, 0.391, 0.392, 0.416], :name=>"sangenya_5cm", :mode=>"markers+lines"}, {:y=>[-0.00145, -0.0013000000000000002, -0.0011, -0.0009000000000000001, -0.0007, -0.0005, -0.00031, 0], :x=>[0.144, 0.212, 0.263, 0.298, 0.321, 0.341, 0.355, 0.374], :name=>"sangenya_15cm", :mode=>"markers+lines"}, {:y=>[-0.0012, -0.001, -0.000804, -0.000616, -0.00041, -0.000295, 0], :x=>[0.119, 0.203, 0.27, 0.303, 0.328, 0.339, 0.361], :name=>"sangenya_25cm", :mode=>"markers+lines"}], @layout={:xaxis=>{:title=>"電圧(V)"}, :yaxis=>{:title=>"電流(I)"}, :font=>{:size=>18}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fc6d91185a8 @id="ba40befe-b30e-4bee-8d22-fd08308c78d1", @data=[{:y=>[-0.010400000000000001, -0.0079, -0.0037, -0.0025, -0.0018000000000000002, -0.0011, -0.00071, -0.00044, -0.00022, -0.00017, -6.8e-05, -2.8e-05, -1.4e-05, -7.4e-06, -2.0e-06, -1.0e-06, -5.0e-07, -3.0e-07, 0, 6.3e-06, 5.7e-05, 0.000173, 0.00041, 0.0006, 0.0013000000000000002, 0.0018000000000000002, 0.0025, 0.0034, 0.0039, 0.0048, 0.006200000000000001, 0.008199999999999999], :x=>[-1.708, -1.626, -1.4, -1.296, -1.209, -1.111, -1.018, -0.929, -0.814, -0.744, -0.62, -0.499, -0.396, -0.308, -0.115, -0.0594, -0.0292, -0.013800000000000002, -0.0009000000000000001, 0.13, 0.272, 0.342, 0.408, 0.417, 0.479, 0.506, 0.54, 0.572, 0.59, 0.617, 0.651, 0.693], :name=>"nagura_dark", :mode=>"markers+lines"}, {:y=>[-0.0018000000000000002, -0.0017, -0.0015, -0.0013000000000000002, -0.0011, -0.0009, -0.0007, -0.0005, 0, -0.0003], :x=>[0.18, 0.194, 0.224, 0.247, 0.27, 0.293, 0.31, 0.331, 0.344, 0.35], :name=>"nagura_5cm", :mode=>"markers+lines"}, {:y=>[-0.00094, -0.00086, -0.0007, -0.00055, -0.0004, -0.00026, 0], :x=>[0.094, 0.154, 0.213, 0.252, 0.278, 0.297, 0.32], :name=>"nagura_15cm", :mode=>"markers+lines"}, {:y=>[-0.000466, -0.00045, -0.00041, -0.00037, -0.00033, -0.00029, -0.000245, 0], :x=>[4.7e-05, 0.121, 0.187, 0.222, 0.249, 0.267, 0.282, 0.339], :name=>"nagura_25cm", :mode=>"markers+lines"}], @layout={:xaxis=>{:title=>"電圧(V)"}, :yaxis=>{:title=>"電流(I)"}, :font=>{:size=>18}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

In [15]:
['nakagawa'].each do |member|
  Daru::DataFrame.from_csv("data/#{member}/dark_solar_battery.csv").tap do |df|
    line = df.filter_rows { |row| -4 < row['v'] && row['v'] < 0.3 }
    f = LinearFitting.new(line['v'].to_a, line['i'].to_a)
    f.fit
    df.map_rows! do |row|
      row['i'] = row['i'] - (f.a * row['v'] + f.b)
      row
    end
    puts member
    df.write_csv("data/#{member}/dark_solar_battery_without_parallel_resistance.csv")
    df.plot(x: 'v', y: 'i', mode: [:markers, :lines], layout: { xaxis: { title: '電圧(V)' }, yaxis: { title: '電流(I)' }, font: { size: 18 } }).show
  end
end
nil

nakagawa


#<Plotly::Offline::HTML:0x007fc6da3e7da0 @id="43897531-56fa-4dc6-ab10-a7bcd7bbe9bb", @data=[{:y=>[0.003168390550630013, 0.0025028296659537035, 0.0018453528887023458, 0.0009001655901236891, 0.00010979099296637256, 1.3108338937442739e-05, -2.1892166592389975e-05, -3.175232457068751e-05, 1.796698032624131e-05, 1.109753324848601e-05, -1.6799781127643104e-05, -1.8617683181529755e-05, -2.1608394082071663e-05, -1.894022978514518e-05, -1.047314605711453e-05, -9.281657905576583e-06, -9.197550480621749e-06, -6.89388539382182e-06, -5.842542581886365e-06, -4.2680166579102743e-07, 3.1291856958734357e-06, 4.040282062239635e-06, 1.0495711616977273e-06, 4.740457906043287e-05, 6.750297859710232e-06, 8.852983483581198e-06, 3.245102455772301e-05, 7.927149848902318e-05, -6.217993911312135e-05], :x=>[0.844, 0.815, 0.785, 0.704, 0.542, 0.455, 0.348, 0.275, 0.207, 0.146, 0.121, 0.0903, 0.0783, 0.06559999999999999, 0.022, 0.0007, -0.0003, -0.0158, -0.028300000000000002, -0.0594, -0.07790000000000001, -0.103, -0.115, -0.143, -0.159, -0.184, -0.203, -0.617, -1.218], :name=>"i", :mode=>"markers+lines", :type=>:scatter}], @layout={:xaxis=>{:title=>"電圧(V)"}, :yaxis=>{:title=>"電流(I)"}, :width=>1000, :height=>500, :font=>{:size=>18}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

In [34]:
a = proc do |row|
  0.2 < row['v']
end
b = proc do |row|
  row['v'] < 0.3 || 0.6 < row['v']
end
  
line_conditions = {
  'data/sangenya/dark_solar_battery.csv' => a,
  'data/nakagawa/dark_solar_battery_without_parallel_resistance.csv' => b,
}
['data/sangenya/dark_solar_battery.csv', 'data/nakagawa/dark_solar_battery_without_parallel_resistance.csv'].map do |filename|
  df = Daru::DataFrame.from_csv(filename).filter_rows { |row| row['v'] > 0 && row['i'] > 0}
  line = df.filter_rows &line_conditions[filename]
  fit = ExpFitting.new(line['v'].to_a, line['i'].to_a)
  fit.fit
    df.plot(x: 'v', y: 'i', layout: { xaxis: { title: '電圧(V)' }, yaxis: { title: '電流(I)', type: :log }, font: { size: 18 } }, mode: [:markers, :lines]).tap do |plot|
    plot.data.push fit.points(0.1, 1)
    plot.show
  end
end
nil

#<Plotly::Offline::HTML:0x007f895cdd94a0 @id="2bbb23e4-ac7c-43d7-9e20-5d1a6000aeea", @data=[{:x=>[0.105, 0.123, 0.165, 0.19, 0.267, 0.311, 0.339, 0.39, 0.418, 0.44, 0.459, 0.486, 0.505, 0.519, 0.538, 0.554, 0.565, 0.582], :y=>[8.0e-07, 9.6e-05, 0.00023, 0.000226, 0.00041, 0.00059, 0.00077, 0.0013000000000000002, 0.0018000000000000002, 0.0023, 0.0029, 0.0039, 0.0048, 0.0055, 0.006500000000000001, 0.0075, 0.0083, 0.0094], :type=>:scatter, :mode=>"markers+lines", :name=>"i"}, {:x=>[0.1, 1], :y=>[6.804330004069051e-05, 0.7694701268873473], :name=>"log y = 1.04e+01 x - 1.06e+01"}], @layout={:width=>1000, :height=>500, :xaxis=>{:title=>"電圧(V)"}, :yaxis=>{:title=>"電流(I)", :type=>:log}, :font=>{:size=>18}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007f895d454870 @id="5297e266-893a-46f5-b8d5-6b5cb54acc6e", @data=[{:x=>[0.844, 0.815, 0.785, 0.704, 0.542, 0.455, 0.207, 0.146], :y=>[0.003168390550630013, 0.0025028296659537035, 0.0018453528887023458, 0.0009001655901236891, 0.00010979099296637256, 1.3108338937442739e-05, 1.796698032624131e-05, 1.109753324848601e-05], :type=>:scatter, :mode=>"markers+lines", :name=>"i"}, {:x=>[0.1, 1], :y=>[7.558740921005443e-06, 0.01064028025865234], :name=>"log y = 8.06e+00 x - 1.26e+01"}], @layout={:width=>1000, :height=>500, :xaxis=>{:title=>"電圧(V)"}, :yaxis=>{:title=>"電流(I)", :type=>:log}, :font=>{:size=>18}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

In [35]:
k = 1.38064852 * 10**(-23)
T = 300
e = 1.60217662 * 10**(-19)
puts "sangenya: n = #{e/(10.4 * k * T)}"
puts "nakagawa: n = #{e/(8.06 * k * T)}"

<main>:1: warning: already initialized constant T
<main>:1: warning: previous definition of T was here


sangenya: n = 3.719398094425201
nakagawa: n = 4.7992233476454205


# 課題2

## Voc

In [125]:
members = ['sangenya', 'nagura', 'nakagawa', 'itano']
names = ['5cm', '15cm', '25cm']

data_hash = names.map do |name|
  arr = members.map do |member|
    df = Daru::DataFrame.from_csv("data/#{member}/#{name}_solar_battery.csv")
    ('%.3f' % df['v'].max)
  end
  [name + '(V)', arr]
end.to_h

voc = Daru::DataFrame.new({'name' => members}.merge(data_hash), order: (['name'] + names.map { |s| s + '(V)' }) )
voc.write_csv('../report/resources/solar/kadai2/voc.csv')

## Isc

In [126]:
members = ['sangenya', 'nagura', 'nakagawa', 'itano']
names = ['5cm', '15cm', '25cm']

data_hash = names.map do |name|
  arr = members.map do |member|
    df = Daru::DataFrame.from_csv("data/#{member}/#{name}_solar_battery.csv")
    ('%.2e' % df.sort(['v'])['i'].to_a.first.abs)
  end
  [name + '(I)', arr]
end.to_h

isc = Daru::DataFrame.new({'name' => members}.merge(data_hash), order: (['name'] + names.map { |s| s + '(I)' }) )
isc.write_csv('../report/resources/solar/kadai2/isc.csv')

## 最適動作点

In [127]:
members = ['sangenya', 'nagura', 'nakagawa', 'itano']
names = ['5cm', '15cm', '25cm']

data_hash = names.map do |name|
  arr = members.map do |member|
    opt_v, opt_i, _ = Daru::DataFrame.from_csv("data/#{member}/#{name}_solar_battery.csv").map_rows do |row|
      [row['v'], row['i'], (row['v'] * row['i']).abs]
    end.max_by { |v,i,w| w }
    "(#{'%.3f' % opt_v}、#{'%.2e' % opt_i})"
  end
  [name + '(V、I)', arr]
end.to_h

optimum = Daru::DataFrame.new({'name' => members}.merge(data_hash), order: (['name'] + names.map { |s| s + '(V、I)' }) )
optimum.write_csv('../report/resources/solar/kadai2/optimum.csv')

## 変換効率

In [161]:
members = ['sangenya', 'nagura', 'nakagawa', 'itano']
names = ['5cm', '15cm', '25cm']
area = 0.8**2 # 0.8cm の 2乗
power = { # W not mW
  '5cm' => (105 * 10**(-3)) * area,
  '15cm' => (42.5 * 10**(-3)) * area,
  '25cm' => (27 * 10**(-3)) * area,
}

data_hash = names.map do |name|
  arr = members.map do |member|
    _, _, w = Daru::DataFrame.from_csv("data/#{member}/#{name}_solar_battery.csv").map_rows do |row|
      [row['v'], row['i'], (row['v'] * row['i']).abs]
    end.max_by { |v,i,w| w }
    "#{'%.3f' % (w/power[name] * 100)}"
  end
  [name + '(％)', arr]
end.to_h

Daru::DataFrame.new({'name' => members}.merge(data_hash), order: (['name'] + names.map{ |s| s + '(％)' }) )
               .write_csv('../report/resources/solar/kadai2/efficiency.csv')
power

{"5cm"=>0.06720000000000001, "15cm"=>0.02720000000000001, "25cm"=>0.017280000000000004}

## FF

In [159]:
members = ['sangenya', 'nagura', 'nakagawa', 'itano']
names = ['5cm', '15cm', '25cm']

data_hash = names.map do |name|
  arr = members.map do |member|
    _, _, w = Daru::DataFrame.from_csv("data/#{member}/#{name}_solar_battery.csv").map_rows do |row|
      [row['v'], row['i'], (row['v'] * row['i']).abs]
    end.max_by { |v,i,w| w }
    w
  end
  [name, arr]
end.to_h

optimum = Daru::DataFrame.new({'name' => members}.merge(data_hash))

data_hash = names.map do |name|
  arr = members.map do |member|
    p_max = optimum.filter_rows { |row| row['name'] == member }[optimum.vectors.to_a.find{ |vect| vect =~ Regexp.new(name) }].to_a.first
    v = voc.filter_rows { |row| row['name'] == member }[voc.vectors.to_a.find{ |vect| vect =~ Regexp.new(name) }].to_a.first
    i = isc.filter_rows { |row| row['name'] == member }[isc.vectors.to_a.find{ |vect| vect =~ Regexp.new(name) }].to_a.first
    ff = p_max / (v.to_f * i.to_f)
    "#{'%.3f' % ff}"
  end
  [name, arr]
end.to_h

Daru::DataFrame.new({'name' => members}.merge(data_hash), order: (['name'] + names) )
               .write_csv('../report/resources/solar/kadai2/ff.csv')

# 課題3

In [171]:
def map_base_name name
  if name == 'good_15cm'
    '15cm'
  else
    name
  end
end

data = ['dark', '5cm', 'good_15cm', '25cm'].map do |base_name|
  df = Daru::DataFrame.from_csv("../2nd/solar_#{base_name}.csv").sort ['v']
  {
    x: df['v'].to_a, y: df['i'].to_a.map { |x| x.is_a?(String) ? x.to_r.to_f : x }, type: :scatter, mode: :'markers+lines', name: map_base_name(base_name)
  }
end
Plotly::Plot.new(data: data, layout: { xaxis: { title: '電圧(V)' }, yaxis: { title: '電流(I)' }, font: { size: 18 } }).show

#<Plotly::Offline::HTML:0x007f895c037568 @id="101eb6a7-eecc-4573-a469-4fea731261c3", @data=[{:x=>[-1.93, -1.7, -1.189, -0.748, -0.314, 0.001, 0.051000000000000004, 0.12, 0.149, 0.199, 0.243, 0.305, 0.356, 0.4, 0.45, 0.496, 0.548, 0.602, 0.647, 0.7000000000000001, 0.752, 0.799, 0.85, 0.907, 0.9500000000000001, 1.002, 1.055, 1.097, 1.1500000000000001, 1.203, 1.248, 1.295, 1.347, 1.403, 1.453, 1.5, 1.549, 1.6, 1.6480000000000001, 1.703, 1.747, 1.795], :y=>[-8.46e-05, -7.46e-05, -5.2799999999999996e-05, -3.54e-05, -1.74e-05, -9.8e-07, 2.7e-06, 9.0e-06, 1.2199999999999998e-05, 1.8399999999999997e-05, 2.5699999999999998e-05, 3.7e-05, 4.95e-05, 6.379999999999999e-05, 7.890000000000001e-05, 9.82e-05, 0.0001248, 0.0001609, 0.00019969999999999998, 0.0002562, 0.00033099999999999997, 0.000419, 0.0005394, 0.0007152, 0.0008837, 0.00114, 0.001501, 0.001858, 0.002447, 0.003243, 0.004162, 0.005437, 0.007385, 0.010379, 0.014433, 0.019973, 0.028683, 0.04352, 0.06515, 0.10543, 0.1572, 0.238], :type=>:scatter, :mode=>:"markers+lines", :name=>"dark"}, {:x=>[0.721, 1.295, 1.385, 1.423, 1.435, 1.442, 1.45, 1.453, 1.458, 1.462, 1.463, 1.464], :y=>[-0.164, -0.153, -0.1255, -0.0928, -0.0771, -0.06609999999999999, -0.0538, -0.046700000000000005, -0.035500000000000004, -0.027, -0.013300000000000001, -0.019], :type=>:scatter, :mode=>:"markers+lines", :name=>"5cm"}, {:x=>[0.975, 1.25, 1.327, 1.428, 1.48, 1.5110000000000001, 1.532, 1.55, 1.56, 1.568, 1.574], :y=>[-0.115, -0.11800000000000001, -0.11750000000000001, -0.1102, -0.09609999999999999, -0.07909999999999999, -0.06420000000000001, -0.047, -0.0354, -0.0251, -0.0148], :type=>:scatter, :mode=>:"markers+lines", :name=>"15cm"}, {:x=>[0.455, 1.3900000000000001, 1.434, 1.469, 1.494, 1.51, 1.512], :y=>[-0.0618, -0.0514, -0.0431, -0.0325, -0.022, -0.0151, -0.014], :type=>:scatter, :mode=>:"markers+lines", :name=>"25cm"}], @layout={:xaxis=>{:title=>"電圧(V)"}, :yaxis=>{:title=>"電流(I)"}, :font=>{:size=>18}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>